<a href="https://colab.research.google.com/github/rameshavinash94/Customer_Segmentation_Clustering/blob/main/Customer_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Estimating Delivery  date**

**Our Business Problem:**

Predicting Olist Delivery date


#### **IMPORTING LIBRARIES**

In [420]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import plotly.express as px
from sklearn.cluster import AgglomerativeClustering

**Load the datasets**

In [421]:
customers = pd.read_csv('olist_customers_dataset.csv')
order_items = pd.read_csv('olist_order_items_dataset.csv')
order_payment= pd.read_csv('olist_order_payments_dataset.csv')
geolocation = pd.read_csv('olist_geolocation_dataset.csv')
order_reviews = pd.read_csv('olist_order_reviews_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')
product_category_name = pd.read_csv('product_category_name_translation.csv')

The ER/Data Modeling Structure of the Datasource
<div>
<img src='https://i.imgur.com/HRhd2Y0.png'>
</div>


*We won't require all the above datasets for our current business usecase for customer segmentation, so selectiong on the necessary files and carryon the EDA and data wrangling on the datasets.*

In [422]:
location_info = geolocation.groupby('geolocation_zip_code_prefix').max().drop_duplicates().reset_index()

In [423]:
location_info

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1001,-23.549292,-46.633559,são paulo,SP
1,1002,-23.544641,-46.633180,são paulo,SP
2,1003,-23.548901,-46.634862,são paulo,SP
3,1004,-23.549181,-46.634057,são paulo,SP
4,1005,-23.548758,-46.634768,são paulo,SP
...,...,...,...,...,...
18993,99960,-27.953409,-52.016483,charrua,RS
18994,99965,-28.173892,-52.034367,agua santa,RS
18995,99970,-28.341329,-51.870397,ciríaco,RS
18996,99980,-28.386239,-51.779942,david canabarro,RS


In [424]:
customers_location=pd.merge(customers,location_info,left_on='customer_zip_code_prefix',right_on='geolocation_zip_code_prefix').drop(columns=['geolocation_zip_code_prefix','geolocation_city','geolocation_state','customer_unique_id']).rename(columns={'geolocation_lat':'customer_lat','geolocation_lng':'customer_lng'})

In [425]:
customers_location

,customer_id,customer_zip_code_prefix,customer_city,customer_state,customer_lat,customer_lng
0,06b8999e2fba1a1fbc88172c00ba8bc7,14409,franca,SP,-20.468849,-47.382173
1,5dca924cc99eea2dc5ba40d11ec5dd0f,14409,franca,SP,-20.468849,-47.382173
2,661897d4968f1b59bfff74c7eb2eb4fc,14409,franca,SP,-20.468849,-47.382173
3,702b62324327ccba20f1be3465426437,14409,franca,SP,-20.468849,-47.382173
4,bdf997bae7ca819b0415f5174d6b4302,14409,franca,SP,-20.468849,-47.382173
...,...,...,...,...,...,...
99128,0b7a30ba373aeb55cf28add5b5477956,87145,paicandu,PR,-23.501620,-52.150755
99129,ff6b440b2465bfaef05c675efb48cd15,98860,eugenio de castro,RS,-28.525494,-54.144722
99130,3f7029d9b98a47370cd5f3a97adcbccd,5538,sao paulo,SP,-23.587405,-46.736508
99131,d9110683c7a282144e9fc97660026a28,74980,aparecida de goiania,GO,-16.821692,-49.243077


In [426]:
seller_locations=pd.merge(sellers,location_info,left_on='seller_zip_code_prefix',right_on='geolocation_zip_code_prefix').drop(columns=['geolocation_zip_code_prefix','geolocation_city','geolocation_state']).rename(columns={'geolocation_lat':'seller_lat','geolocation_lng':'seller_lng'})

In [427]:
seller_locations

,seller_id,seller_zip_code_prefix,seller_city,seller_state,seller_lat,seller_lng
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP,-22.884842,-47.045846
1,e0eabded302882513ced4ea3eb0c7059,13023,campinas,SP,-22.884842,-47.045846
2,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP,-22.347646,-46.920804
3,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ,-22.864365,-43.170800
4,1d2732ef8321502ee8488e8bed1ab8cd,20031,rio de janeiro,RJ,-22.864365,-43.170800
...,...,...,...,...,...,...
3083,f1fdf2d13186575751aa25876536d85c,5314,sao paulo,SP,-23.524115,-46.732087
3084,98dddbc4601dd4443ca174359b237166,87111,sarandi,PR,-23.434520,-51.859687
3085,74871d19219c7d518d0090283e03c137,4650,sao paulo,SP,-23.655006,-46.669658
3086,e603cf3fec55f8697c9059638d6c8eb5,96080,pelotas,RS,-31.709246,-52.307455


In [428]:
orders

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00
...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00


In [429]:
product_info = pd.merge(order_items,products)

In [430]:
order_product_info= pd.merge(orders,product_info)

In [431]:
customers_orders=pd.merge(customers_location,order_product_info)

In [432]:
final_Df = pd.merge(customers_orders,seller_locations)

In [433]:
final_Df = final_Df[final_Df['order_status']=='delivered']

In [434]:
def haversine_distance(lat1, lon1, lat2, lon2):
   r = 6371
   phi1 = np.radians(lat1)
   phi2 = np.radians(lat2)
   delta_phi = np.radians(np.array(lat2) - np.array(lat1))
   delta_lambda = np.radians(np.array(lon2) - np.array(lon1))
   a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
   res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
   return np.round(res*0.6214, 2)

In [435]:
final_Df['distance']= haversine_distance(final_Df['customer_lat'],final_Df['customer_lng'],final_Df['seller_lat'],final_Df['seller_lng'])

In [436]:
final_Df

,customer_id,customer_zip_code_prefix,customer_city,customer_state,customer_lat,customer_lng,order_id,order_status,order_purchase_timestamp,order_approved_at,...,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,seller_lat,seller_lng,distance
0,06b8999e2fba1a1fbc88172c00ba8bc7,14409,franca,SP,-20.468849,-47.382173,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,...,8683.0,54.0,64.0,31.0,8577,itaquaquecetuba,SP,-23.479473,-46.357851,218.13
1,4e7b3e00288586ebd08712fdd0374a03,1151,sao paulo,SP,-23.527788,-46.652997,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,...,8267.0,52.0,52.0,17.0,8577,itaquaquecetuba,SP,-23.479473,-46.357851,19.00
2,b2b6027bc5c5109e529d4dc6358b12c3,8775,mogi das cruzes,SP,-23.493944,-46.172406,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,...,12160.0,56.0,51.0,28.0,8577,itaquaquecetuba,SP,-23.479473,-46.357851,11.79
3,f2fdb2409d1c2b5018b19767572978d6,30575,belo horizonte,MG,-19.818637,-43.929691,930fb714ff2992c4ff14bf920af830e6,delivered,2017-12-03 14:52:12,2017-12-03 14:59:22,...,11875.0,57.0,55.0,22.0,8577,itaquaquecetuba,SP,-23.479473,-46.357851,297.14
4,bf45a4524fffba803efa07f225cdfda5,20231,rio de janeiro,RJ,-22.908714,-43.182574,22167b7870f3c3b844087c5cbafac521,delivered,2018-03-07 17:56:08,2018-03-09 02:30:55,...,13175.0,57.0,55.0,22.0,8577,itaquaquecetuba,SP,-23.479473,-46.357851,205.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112052,d5d4cab5f11cc0e7d3c3882091d815c7,17320,mineiros do tiete,SP,-22.404782,-48.443557,deef94d95b5cf9c23e10621ad71a96b2,delivered,2017-03-18 11:40:26,2017-03-18 11:40:26,...,800.0,28.0,12.0,18.0,4164,sao paulo,SP,-23.621342,-46.605415,143.98
112053,1d3e643209846ffadac771cc5dc1ed4b,79320,corumba,MS,-19.014518,-57.643125,fd0342fda6ba083712b6ed0d4e6ac483,delivered,2018-07-23 16:24:27,2018-07-23 17:21:07,...,1450.0,60.0,5.0,30.0,83323,pinhais,PR,-25.432332,-49.183859,699.20
112054,124c96fb17eb2fc7f1611d3ca83d11ea,6887,itapecerica da serra,SP,-23.670573,-46.812776,90dbcfe5c30e1fd81cfaf22dc879e81b,delivered,2018-08-17 10:15:31,2018-08-17 10:30:16,...,2200.0,16.0,35.0,49.0,9380,maua,SP,-23.624618,-46.451878,23.06
112055,6d0c650bd30f4160b84c3634dd05b509,17533,marilia,SP,-22.154833,-49.935832,1175cf55b833823e2478956d6bb6e43c,delivered,2017-06-23 07:15:39,2017-06-23 07:25:15,...,700.0,16.0,16.0,16.0,87015,maringa,PR,-23.418061,-51.882767,151.66


In [437]:
final_Df.columns

Index(['customer_id', 'customer_zip_code_prefix', 'customer_city',
       'customer_state', 'customer_lat', 'customer_lng', 'order_id',
       'order_status', 'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'order_item_id', 'product_id',
       'seller_id', 'shipping_limit_date', 'price', 'freight_value',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'seller_zip_code_prefix', 'seller_city', 'seller_state', 'seller_lat',
       'seller_lng', 'distance'],
      dtype='object')

In [438]:
final_Df[['order_delivered_customer_date','order_purchase_timestamp','order_estimated_delivery_date','order_approved_at','order_delivered_carrier_date','shipping_limit_date']] = final_Df[['order_delivered_customer_date','order_purchase_timestamp','order_estimated_delivery_date','order_approved_at','order_delivered_carrier_date','shipping_limit_date']].apply(pd.to_datetime) # conversion required

final_Df['actual_delivery_timeline']=(final_Df['order_delivered_customer_date'] - final_Df['order_purchase_timestamp']).dt.days
final_Df['estimated_delivery_timeline']=(final_Df['order_delivered_customer_date'] - final_Df['order_estimated_delivery_date']).dt.days

final_Df['Approval_timeline']= (final_Df['order_approved_at'] - final_Df['order_purchase_timestamp']).dt.days
final_Df['processing_timeline'] = (final_Df['order_delivered_carrier_date'] - final_Df['order_approved_at']).dt.days
final_Df['processing_timeline'] = (final_Df['order_delivered_carrier_date'] - final_Df['order_approved_at']).dt.days
final_Df['seller_timeline'] =  (final_Df['order_delivered_carrier_date'] - final_Df['shipping_limit_date']).dt.days

The formula to determine the density of an item is: LxWxH / 1728 = Cubic Feet (CU FT), Weight (Pounds)/CU FT = Density (#PCF)

In [439]:
def density(l,w,h,weight):
    l=np.array(l)*0.393701 # to convert to inches
    w=np.array(w)*0.393701
    h=np.array(h)*0.393701
    Cubic_feet = l*w*h/1728
    Density = (np.array(weight)*0.00220462)/Cubic_feet
    return np.round(Density,2)

In [440]:
final_Df['pkg_density']=density(final_Df['product_length_cm'],final_Df['product_width_cm'],final_Df['product_height_cm'],final_Df['product_weight_g'])

In [441]:
final_Df

,customer_id,customer_zip_code_prefix,customer_city,customer_state,customer_lat,customer_lng,order_id,order_status,order_purchase_timestamp,order_approved_at,...,seller_state,seller_lat,seller_lng,distance,actual_delivery_timeline,estimated_delivery_timeline,Approval_timeline,processing_timeline,seller_timeline,pkg_density
0,06b8999e2fba1a1fbc88172c00ba8bc7,14409,franca,SP,-20.468849,-47.382173,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,...,SP,-23.479473,-46.357851,218.13,8.0,-11.0,0.0,6.0,0.0,5.06
1,4e7b3e00288586ebd08712fdd0374a03,1151,sao paulo,SP,-23.527788,-46.652997,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,...,SP,-23.479473,-46.357851,19.00,26.0,1.0,1.0,21.0,5.0,11.23
2,b2b6027bc5c5109e529d4dc6358b12c3,8775,mogi das cruzes,SP,-23.493944,-46.172406,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,...,SP,-23.479473,-46.357851,11.79,14.0,-13.0,0.0,14.0,0.0,9.49
3,f2fdb2409d1c2b5018b19767572978d6,30575,belo horizonte,MG,-19.818637,-43.929691,930fb714ff2992c4ff14bf920af830e6,delivered,2017-12-03 14:52:12,2017-12-03 14:59:22,...,SP,-23.479473,-46.357851,297.14,25.0,-14.0,0.0,9.0,-1.0,10.75
4,bf45a4524fffba803efa07f225cdfda5,20231,rio de janeiro,RJ,-22.908714,-43.182574,22167b7870f3c3b844087c5cbafac521,delivered,2018-03-07 17:56:08,2018-03-09 02:30:55,...,SP,-23.479473,-46.357851,205.48,18.0,-29.0,1.0,11.0,-3.0,11.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112052,d5d4cab5f11cc0e7d3c3882091d815c7,17320,mineiros do tiete,SP,-22.404782,-48.443557,deef94d95b5cf9c23e10621ad71a96b2,delivered,2017-03-18 11:40:26,2017-03-18 11:40:26,...,SP,-23.621342,-46.605415,143.98,9.0,-10.0,0.0,3.0,-2.0,8.26
112053,1d3e643209846ffadac771cc5dc1ed4b,79320,corumba,MS,-19.014518,-57.643125,fd0342fda6ba083712b6ed0d4e6ac483,delivered,2018-07-23 16:24:27,2018-07-23 17:21:07,...,PR,-25.432332,-49.183859,699.20,9.0,-13.0,0.0,0.0,-4.0,10.06
112054,124c96fb17eb2fc7f1611d3ca83d11ea,6887,itapecerica da serra,SP,-23.670573,-46.812776,90dbcfe5c30e1fd81cfaf22dc879e81b,delivered,2018-08-17 10:15:31,2018-08-17 10:30:16,...,SP,-23.624618,-46.451878,23.06,4.0,-6.0,0.0,3.0,-3.0,5.01
112055,6d0c650bd30f4160b84c3634dd05b509,17533,marilia,SP,-22.154833,-49.935832,1175cf55b833823e2478956d6bb6e43c,delivered,2017-06-23 07:15:39,2017-06-23 07:25:15,...,PR,-23.418061,-51.882767,151.66,11.0,-14.0,0.0,4.0,-3.0,10.67


In [442]:
regression_test = final_Df.drop(columns=['order_purchase_timestamp','order_approved_at','order_approved_at','order_delivered_carrier_date','order_delivered_customer_date','order_estimated_delivery_date','seller_id', 'shipping_limit_date','product_category_name', 'product_name_lenght','product_description_lenght', 'product_photos_qty', 'product_weight_g','product_length_cm', 'product_height_cm', 'product_width_cm','customer_city','customer_state', 'customer_lat', 'customer_lng','seller_city', 'seller_state', 'seller_lat','seller_lng','order_status','customer_id','order_id','order_item_id','product_id'])

In [443]:
regression_test

,customer_zip_code_prefix,price,freight_value,seller_zip_code_prefix,distance,actual_delivery_timeline,estimated_delivery_timeline,Approval_timeline,processing_timeline,seller_timeline,pkg_density
0,14409,124.99,21.88,8577,218.13,8.0,-11.0,0.0,6.0,0.0,5.06
1,1151,139.94,17.79,8577,19.00,26.0,1.0,1.0,21.0,5.0,11.23
2,8775,149.94,23.36,8577,11.79,14.0,-13.0,0.0,14.0,0.0,9.49
3,30575,169.99,36.85,8577,297.14,25.0,-14.0,0.0,9.0,-1.0,10.75
4,20231,146.99,40.42,8577,205.48,18.0,-29.0,1.0,11.0,-3.0,11.93
...,...,...,...,...,...,...,...,...,...,...,...
112052,17320,98.00,12.08,4164,143.98,9.0,-10.0,0.0,3.0,-2.0,8.26
112053,79320,75.00,27.48,83323,699.20,9.0,-13.0,0.0,0.0,-4.0,10.06
112054,6887,159.90,12.14,9380,23.06,4.0,-6.0,0.0,3.0,-3.0,5.01
112055,17533,49.70,16.11,87015,151.66,11.0,-14.0,0.0,4.0,-3.0,10.67


In [444]:
regression_test.corr()

,customer_zip_code_prefix,price,freight_value,seller_zip_code_prefix,distance,actual_delivery_timeline,estimated_delivery_timeline,Approval_timeline,processing_timeline,seller_timeline,pkg_density
customer_zip_code_prefix,1.000000,0.041957,0.226805,0.076250,0.478067,0.273213,-0.074389,0.032433,0.002481,-0.003779,-0.000493
price,0.041957,1.000000,0.412948,0.087006,0.084289,0.062688,-0.003248,0.010179,0.062340,0.010393,0.038015
freight_value,0.226805,0.412948,1.000000,0.150778,0.384281,0.214686,-0.040336,0.024747,0.085163,0.003564,0.042476
seller_zip_code_prefix,0.076250,0.087006,0.150778,1.000000,0.168328,0.048840,-0.117137,-0.005066,0.007522,-0.006434,0.013349
distance,0.478067,0.084289,0.384281,0.168328,1.000000,0.389923,-0.073259,0.026530,0.014311,0.000932,0.005508
actual_delivery_timeline,0.273213,0.062688,0.214686,0.048840,0.389923,1.000000,0.596709,0.077315,0.395511,0.145818,0.015084
estimated_delivery_timeline,-0.074389,-0.003248,-0.040336,-0.117137,-0.073259,0.596709,1.000000,0.035207,0.193081,0.249841,0.008086
Approval_timeline,0.032433,0.010179,0.024747,-0.005066,0.026530,0.077315,0.035207,1.000000,-0.047243,-0.015467,-0.001271
processing_timeline,0.002481,0.062340,0.085163,0.007522,0.014311,0.395511,0.193081,-0.047243,1.000000,0.411571,-0.001133
seller_timeline,-0.003779,0.010393,0.003564,-0.006434,0.000932,0.145818,0.249841,-0.015467,0.411571,1.000000,-0.002111


In [454]:
data_wrangler = regression_test.dropna().drop_duplicates()

In [455]:
X = data_wrangler.drop(columns=['actual_delivery_timeline'])
y=data_wrangler['actual_delivery_timeline']

In [456]:
X.columns

Index(['customer_zip_code_prefix', 'price', 'freight_value',
       'seller_zip_code_prefix', 'distance', 'estimated_delivery_timeline',
       'Approval_timeline', 'processing_timeline', 'seller_timeline',
       'pkg_density'],
      dtype='object')

In [457]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_values=scaler.fit_transform(X)
scaler_df = pd.DataFrame(scaled_values,columns=X.columns)

In [461]:
scaler_df

,customer_zip_code_prefix,price,freight_value,seller_zip_code_prefix,distance,estimated_delivery_timeline,Approval_timeline,processing_timeline,seller_timeline,pkg_density
0,-0.692333,0.005607,0.114070,-0.579459,-0.418634,0.096411,-0.362501,1.049407,0.768280,-0.156024
1,-1.137032,0.085147,-0.144619,-0.579459,-0.947472,1.271743,1.007316,5.290576,1.767968,-0.015985
2,-0.881308,0.138351,0.207679,-0.579459,-0.966620,-0.099477,-0.362501,3.311364,0.768280,-0.055478
3,-0.150095,0.245024,1.060910,-0.579459,-0.208805,-0.197422,-0.362501,1.897641,0.568343,-0.026880
4,-0.497052,0.122656,1.286710,-0.579459,-0.452230,-1.666586,1.007316,2.463130,0.168467,-0.000097
...,...,...,...,...,...,...,...,...,...,...
99347,-0.594693,-0.137989,-0.505772,-0.738863,-0.615557,0.194356,-0.362501,0.201173,0.368405,-0.083395
99348,1.484906,-0.260358,0.468265,2.120472,0.858962,-0.099477,-0.362501,-0.647061,-0.031470,-0.042540
99349,-0.944636,0.191342,-0.501977,-0.550454,-0.936689,0.586133,-0.362501,0.201173,0.168467,-0.157159
99350,-0.587548,-0.394963,-0.250878,2.253832,-0.595161,-0.197422,-0.362501,0.483918,0.168467,-0.028695


In [ ]:
# Q1 = scaler_df.density.quantile(0.05)
# Q3 = scaler_df.density.quantile(0.95)
# IQR = Q3 - Q1
# scaler_df = scaler_df[(scaler_df.density >= Q1 - 1.5*IQR) & (scaler_df.density <= Q3 + 1.5*IQR)]

# Q1 = scaler_df.distance.quantile(0.05)
# Q3 = scaler_df.distance.quantile(0.95)
# IQR = Q3 - Q1
# scaler_df = scaler_df[(scaler_df.distance >= Q1 - 1.5*IQR) & (scaler_df.distance <= Q3 + 1.5*IQR)]

In [ ]:
# #plotting post removing outliers
# attributes = ['distance','density']
# plt.rcParams['figure.figsize'] = [10,8]
# sns.boxplot(data = model_df[attributes], orient="v", palette="Set2" ,whis=1.5,saturation=1, width=0.7)
# plt.title("Variable Distribution post Removing Outliers ", fontsize = 14, fontweight = 'bold')
# plt.ylabel("Range", fontweight = 'bold')
# plt.xlabel("Attributes", fontweight = 'bold')
# plt.show()

**TRAIN TEST SPLIT**

In [468]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(scaler_df, y, test_size=0.2, random_state=42)

**LINEAR REGRESSION**

In [485]:
from sklearn.linear_model import LinearRegression
# creating an object of LinearRegression class
LR = LinearRegression()
# fitting the training data
LR.fit(X_train,y_train)

LinearRegression()

In [484]:
y_prediction =  LR.predict(X_test)
y_prediction

array([12.12123296,  9.11220876,  6.90677025, ..., -8.95894015,
        9.07020243, 22.09408716])

In [486]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
# predicting the accuracy score
score=r2_score(y_test,y_prediction)
print("r2 score is",score)
print("mean_sqrd_error is=",mean_squared_error(y_test,y_prediction))
print("root_mean_squared error of is=",np.sqrt(mean_squared_error(y_test,y_prediction)))

r2 score is 0.6590373599887461
mean_sqrd_error is= 30.31012412061343
root_mean_squared error of is= 5.505463115907093


**LASSO REGRESSION**

In [483]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)
lasso_pred = lasso.predict(X_test)
lasso.score(X_test, y_test)

0.6565011266259855

In [487]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
# predicting the accuracy score
score=r2_score(y_test,lasso_pred)
print("r2 score is",score)
print("mean_sqrd_error is=",mean_squared_error(y_test,lasso_pred))
print("root_mean_squared error of is=",np.sqrt(mean_squared_error(y_test,lasso_pred)))

r2 score is 0.6565011266259855
mean_sqrd_error is= 30.535584446769928
root_mean_squared error of is= 5.525901233895691


**Ridge Regression**

In [488]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=0.1)
ridge.fit(X_train, y_train)
ridge_pred = ridge.predict(X_test)
ridge.score(X_test, y_test)

0.6590373452446766

In [489]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
# predicting the accuracy score
score=r2_score(y_test,ridge_pred)
print("r2 score is",score)
print("mean_sqrd_error is=",mean_squared_error(y_test,ridge_pred))
print("root_mean_squared error of is=",np.sqrt(mean_squared_error(y_test,ridge_pred)))

r2 score is 0.6590373452446766
mean_sqrd_error is= 30.31012543129831
root_mean_squared error of is= 5.505463234942026


In [471]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
# predicting the accuracy score
score=r2_score(y_test,y_prediction)
print("r2 score is",score)
print("mean_sqrd_error is=",mean_squared_error(y_test,y_prediction))
print("root_mean_squared error of is=",np.sqrt(mean_squared_error(y_test,y_prediction)))

r2 score is 0.6590373599887461
mean_sqrd_error is= 30.31012412061343
root_mean_squared error of is= 5.505463115907093


**POLYNOMIAL REGRESSION**

In [495]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
degree=2
polyreg=make_pipeline(PolynomialFeatures(degree),LinearRegression())
polyreg.fit(X_train,y_train)

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures()),
                ('linearregression', LinearRegression())])

In [496]:
y_prediction =  polyreg.predict(X_test)
y_prediction

array([12.25263148,  6.2558147 ,  5.63406002, ...,  2.11446912,
        9.98462925, 21.17514324])

In [497]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
# predicting the accuracy score
score=r2_score(y_test,y_prediction)
print("r2 score is",score)
print("mean_sqrd_error is=",mean_squared_error(y_test,y_prediction))
print("root_mean_squared error of is=",np.sqrt(mean_squared_error(y_test,y_prediction)))

r2 score is 0.7687723842707918
mean_sqrd_error is= 20.55514860113261
root_mean_squared error of is= 4.533778622863341


**NEURAL NETWORK**

In [499]:
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression

regr = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)

array([ 9.48075682,  5.25047394,  6.86902966, ..., 11.91555863,
        9.72945846, 18.93477   ])

In [500]:
y_prediction = regr.predict(X_test)

In [501]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
# predicting the accuracy score
score=r2_score(y_test,y_prediction)
print("r2 score is",score)
print("mean_sqrd_error is=",mean_squared_error(y_test,y_prediction))
print("root_mean_squared error of is=",np.sqrt(mean_squared_error(y_test,y_prediction)))

r2 score is 0.8583010970335043
mean_sqrd_error is= 12.596427973831632
root_mean_squared error of is= 3.54914468200884


**Support Vector Regression**

In [502]:
from sklearn.svm import SVR
svr = SVR().fit(X_train, y_train)

In [504]:
y_prediction = svr.predict(X_test)

In [505]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
# predicting the accuracy score
score=r2_score(y_test,y_prediction)
print("r2 score is",score)
print("mean_sqrd_error is=",mean_squared_error(y_test,y_prediction))
print("root_mean_squared error of is=",np.sqrt(mean_squared_error(y_test,y_prediction)))

r2 score is 0.7269795688021208
mean_sqrd_error is= 24.27035160449832
root_mean_squared error of is= 4.926494859887537


**Random Forest Regressor**

In [508]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(max_depth=7).fit(X_train, y_train)
y_prediction = rfr.predict(X_test)

In [509]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
# predicting the accuracy score
score=r2_score(y_test,y_prediction)
print("r2 score is",score)
print("mean_sqrd_error is=",mean_squared_error(y_test,y_prediction))
print("root_mean_squared error of is=",np.sqrt(mean_squared_error(y_test,y_prediction)))

r2 score is 0.8143859890051436
mean_sqrd_error is= 16.500293731868428
root_mean_squared error of is= 4.062055358050704
